# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

In [1]:
llm_config = {
        "model": "qwen2",
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama",
    }

In [2]:
from autogen import ConversableAgent

## Creating the needed agents

##### Optimize the system message as per the model, Using OpenAI it's working great

In [3]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

##### Asking customer's interest on the topic
 We will use human_input_mode="NEVER": we used LLM as human_input_mode="NEVER"<br>
 Update it as per the requirement as per the use case 

In [4]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

Responds jokes or information as per the customer interest

In [6]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

Human input as Always it will take human feedback

In [7]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [8]:
chats = [
    {
        # sends initial message to kick of the agent
        "sender": onboarding_personal_information_agent,
        
        # receives sends response to the recipient
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        # In sequential chat we may want to sequential message
        # in here we are using reflection_with_llm to summary the message from 1st to previous
        "summary_method": "reflection_with_llm",
        
        # retrieving the name and location from history
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        
        # runs till max_tuns reached
        "max_turns": 2,
        "clear_history" : True
    },
    {
        # sending message from onboarding top agent to customer proxy agent
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        # you can increase as per your requirements
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process
**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [10]:
from autogen import initiate_chats
# starting first chat
# it would ask for name and location
chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


/Users/rajagopalvajja/PycharmProjects/Experiments/.venv/lib/python3.12/site-packages/autogen/agentchat/chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


customer_proxy_agent (to Onboarding Personal Information Agent):

Tom

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Could you please share your location as well, Tom?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

Los Angeles

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{"name": "Tom", "location": "Los Angeles"}
{"name": "Tom", "location": "Los Angeles"}

--------------------------------------------------------------------

## Print out the summary

In [11]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{"name": "Tom", "location": "Los Angeles"}


The user, Tom from Los Angeles, is interested in reading about topics related to Disney.


The conversation summary focuses on recommendations tailored to a Disney enthusiast named Tom in Los Angeles, including fun facts about Disneyland Paris, a playful joke related to Mickey Mouse and Pluto, and an image of experiencing Disney magic at Walt Disney World.




## Print out the cost

In [12]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0, 'qwen2': {'cost': 0, 'prompt_tokens': 370, 'completion_tokens': 52, 'total_tokens': 422}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'qwen2': {'cost': 0, 'prompt_tokens': 185, 'completion_tokens': 26, 'total_tokens': 211}}}


{'usage_including_cached_inference': {'total_cost': 0, 'qwen2': {'cost': 0, 'prompt_tokens': 175, 'completion_tokens': 79, 'total_tokens': 254}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'qwen2': {'cost': 0, 'prompt_tokens': 175, 'completion_tokens': 79, 'total_tokens': 254}}}


{'usage_including_cached_inference': {'total_cost': 0, 'qwen2': {'cost': 0, 'prompt_tokens': 945, 'completion_tokens': 452, 'total_tokens': 1397}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'qwen2': {'cost': 0, 'prompt_tokens': 945, 'completion_tokens': 452, 'total_tokens': 1397}}}


